In [2]:
import torch
import torch.nn as nn
from models.dpt import DPTSegmentationWithCarbon
from dataset import CarbonDataset
from torch.utils.data import DataLoader
from torchvision import transforms
from models.util import calculate_correlation
kwargs = {
    "num_classes": 7,
}

model = DPTSegmentationWithCarbon(num_classes=kwargs["num_classes"])
path = "Dataset/Training/image/SN10_Forest_IMAGE"
transform = transforms.Compose([transforms.Resize((256, 256)), transforms.ToTensor()])
data_loader = DataLoader(CarbonDataset(path,transform, mode = "Train"), batch_size=2, shuffle=True)

x, carbon,gt = data_loader.__iter__().__next__()
gt_pred, carbon_pred = model(x)
corr = calculate_correlation(gt_pred, gt)
print(corr)


/Users/seungwoo/anaconda3/envs/transformer/lib/python3.12/site-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name vit_base_resnet50_384 to current vit_base_r50_s16_384.orig_in21k_ft_in1k.
  model = create_fn(


Dataset/Training/image/SN10_Forest_IMAGE Done.
Dataset/Training/image/SN10_Forest_SH Done.
Dataset/Training/label/SN10_Forest_Carbon Done.
Dataset/Training/label/SN10_Forest_GT Done.


RuntimeError: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.